In [1]:
# imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import gc

In [2]:
matrix_array = np.load('Data/matrix_array_80_normalized.npy')
answer_array = np.load('Data/answer_array_80.npy')

In [3]:
matrix_array.shape, answer_array.shape    

((627703, 80, 19), (627703, 3))

In [4]:
from sklearn.model_selection import train_test_split

X = matrix_array
y = answer_array


X_train, X_temp, y_train, y_temp = train_test_split(X,y, test_size=0.8, random_state=1, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

del matrix_array, answer_array, X, y, X_train ,X_temp, y_train, y_temp, X_valid, y_valid
gc.collect()

0

In [5]:
from datetime import datetime, timezone

# Convert ISO 8601 date strings to Unix timestamp (milliseconds)
def iso_to_unix(iso_str):
    dt = datetime.strptime(iso_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
    return int(dt.timestamp() * 1000)  # Convert to milliseconds

def unix_to_iso(unix_timestamp_ms):
    # Convert milliseconds to seconds
    unix_timestamp_s = unix_timestamp_ms / 1000
    # Create a datetime object from the Unix timestamp
    dt = datetime.utcfromtimestamp(unix_timestamp_s)
    # Format the datetime object as an ISO 8601 date string
    iso_str = dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    return iso_str

In [6]:
# data = pd.read_csv('Data/historical_BTCUSDT_3min_data.csv')
# data_2023 = data[data['Open time']> iso_to_unix('2023-01-01T00:00:00Z')]
# data_2023.to_csv('Data/2023_BTC_data.csv', index=False)
# data_2023 = data_2023.drop(['Close time', 'Ignore'], axis=1)
# data_2023_no_opentime = data_2023.drop(['Open time'], axis=1)
# data_2023

data_2023 = pd.read_csv('Data/2023_BTCUSDT_3min_data.csv')
data_2023 = data_2023.drop(['Ignore', 'Close time'], axis=1)
data_2023_no_opentime = data_2023.drop(['Open time'], axis=1)

data_2024 = pd.read_csv('Data/2024_BTCUSDT_3min_data.csv')
data_2024 = data_2024.drop(['Ignore', 'Close time'], axis=1)
data_2024_no_opentime = data_2024.drop(['Open time'], axis=1)

In [7]:
# Define the window sizes for the moving averages
windows = [5, 10, 20, 30, 60, 120, 240]
# Define the window size and standard deviation multiplier for the Bollinger Bands
BB_window_size = 90
BB_std_multiplier = 1


# ADDING MA
for window in windows:
    # Calculate the moving average
    moving_avg = data_2023['Close'].rolling(window).mean()
    # Add the moving average as a new column to the data_2023
    data_2023[f'MA{window}'] = moving_avg

# Calculate the moving average and standard deviation
BB_moving_avg = data_2023['Close'].rolling(BB_window_size).mean()
BB_std_dev = data_2023['Close'].rolling(BB_window_size).std()

# ADDING BB
# Calculate the Bollinger Bands
BB_upper_band = BB_moving_avg + (BB_std_multiplier * BB_std_dev)
BB_lower_band = BB_moving_avg - (BB_std_multiplier * BB_std_dev)

# Add the Bollinger Bands and moving average as new columns to the data_2023
data_2023[f'MA{BB_window_size}'] = BB_moving_avg
data_2023[f'Upper_Band{BB_window_size}'] = BB_upper_band
data_2023[f'Lower_Band{BB_window_size}'] = BB_lower_band





# drop NaN values
data_2023 = data_2023.dropna().reset_index(drop=True)
data_2023

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,MA5,MA10,MA20,MA30,MA60,MA120,MA240,MA90,Upper_Band90,Lower_Band90
0,1672574220000,16553.35,16557.00,16552.71,16556.66,284.56424,4.711138e+06,8016,140.79799,2.331014e+06,16551.920,16550.547,16548.1790,16545.478000,16536.728000,16532.152083,16531.968667,16531.167667,16544.793901,16517.541432
1,1672574400000,16556.66,16558.05,16554.91,16558.04,207.12723,3.429338e+06,7673,108.88610,1.802822e+06,16553.250,16551.694,16548.7635,16546.038667,16537.489333,16532.317167,16532.058708,16531.639111,16545.453729,16517.824493
2,1672574580000,16557.65,16563.16,16555.21,16556.58,293.15135,4.854459e+06,8439,146.91935,2.432944e+06,16554.820,16552.395,16549.3005,16546.744667,16538.291000,16532.483000,16532.141542,16532.152333,16546.026529,16518.278137
3,1672574760000,16556.58,16559.89,16550.92,16553.17,294.94605,4.882716e+06,9135,136.37729,2.257711e+06,16555.642,16552.862,16549.5900,16547.274333,16539.027167,16532.637083,16532.241083,16532.537000,16546.509407,16518.564593
4,1672574940000,16553.17,16554.41,16548.58,16553.54,190.53334,3.153707e+06,7096,90.21901,1.493346e+06,16555.598,16553.188,16549.6975,16547.865333,16539.732833,16532.773083,16532.356500,16532.869000,16546.982390,16518.755610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174931,1704066480000,42267.89,42273.63,42247.87,42249.06,70.34494,2.972979e+06,1683,39.42697,1.666304e+06,42241.138,42210.400,42229.0150,42325.899333,42449.714167,42534.281167,42529.798000,42504.295889,42661.796127,42346.795651
174932,1704066660000,42249.06,42249.07,42221.22,42228.85,36.70267,1.549980e+06,1372,10.80352,4.562417e+05,42242.046,42219.978,42226.8870,42315.772000,42443.618333,42531.196667,42528.609917,42499.938889,42659.584200,42340.293578
174933,1704066840000,42228.86,42240.93,42222.10,42240.93,30.49513,1.287764e+06,1373,18.94477,7.999862e+05,42245.562,42227.066,42223.7170,42305.496333,42438.500167,42528.170750,42527.442583,42495.418556,42656.583736,42334.253375
174934,1704067020000,42240.92,42283.59,42240.92,42283.58,56.90309,2.405381e+06,1847,34.16134,1.444019e+06,42254.060,42237.244,42225.0020,42295.832333,42433.926333,42525.398667,42526.464333,42491.090889,42652.660964,42329.520814


In [8]:
# Define the window sizes for the moving averages
windows = [5, 10, 20, 30, 60, 120, 240]
# Define the window size and standard deviation multiplier for the Bollinger Bands
BB_window_size = 90
BB_std_multiplier = 1


# ADDING MA
for window in windows:
    # Calculate the moving average
    moving_avg = data_2024['Close'].rolling(window).mean()
    # Add the moving average as a new column to the data_2023
    data_2024[f'MA{window}'] = moving_avg

# Calculate the moving average and standard deviation
BB_moving_avg = data_2024['Close'].rolling(BB_window_size).mean()
BB_std_dev = data_2024['Close'].rolling(BB_window_size).std()

# ADDING BB
# Calculate the Bollinger Bands
BB_upper_band = BB_moving_avg + (BB_std_multiplier * BB_std_dev)
BB_lower_band = BB_moving_avg - (BB_std_multiplier * BB_std_dev)

# Add the Bollinger Bands and moving average as new columns to the data_2023
data_2024[f'MA{BB_window_size}'] = BB_moving_avg
data_2024[f'Upper_Band{BB_window_size}'] = BB_upper_band
data_2024[f'Lower_Band{BB_window_size}'] = BB_lower_band





# drop NaN values
data_2024 = data_2024.dropna().reset_index(drop=True)
data_2024

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,MA5,MA10,MA20,MA30,MA60,MA120,MA240,MA90,Upper_Band90,Lower_Band90
0,1704110220000,42699.98,42699.99,42690.20,42690.20,17.11771,7.308644e+05,1164,11.49433,4.907639e+05,42661.220,42643.637,42677.0035,42685.296333,42659.149167,42547.639333,42492.441625,42604.693556,42693.105219,42516.281892
1,1704110400000,42690.21,42739.22,42690.20,42709.20,49.46675,2.113035e+06,3419,34.70596,1.482377e+06,42677.110,42652.034,42675.4085,42686.722667,42661.478333,42551.540167,42494.040375,42607.039111,42695.391295,42518.686927
2,1704110580000,42709.19,42709.20,42691.10,42693.44,18.60200,7.942602e+05,1682,6.19812,2.646414e+05,42686.954,42656.056,42672.7600,42686.891000,42663.819667,42554.826500,42495.224708,42609.241556,42697.244919,42521.238192
3,1704110760000,42693.44,42693.45,42622.00,42625.99,54.82226,2.338596e+06,2040,13.06340,5.572799e+05,42683.764,42655.655,42666.1615,42684.931000,42665.104000,42557.624917,42496.082708,42610.663667,42697.879819,42523.447514
4,1704110940000,42625.99,42653.39,42625.99,42642.92,13.25919,5.654427e+05,1402,7.72162,3.292666e+05,42672.350,42658.546,42661.0675,42683.028000,42666.686500,42560.630000,42496.764375,42612.674333,42698.498581,42526.850086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47277,1712620080000,71636.00,71683.82,71635.99,71676.45,22.69487,1.626456e+06,2027,10.81796,7.752474e+05,71677.156,71748.108,71800.5420,71759.818333,71740.952333,71761.161333,71834.800167,71726.212444,71818.054794,71634.370095
47278,1712620260000,71676.45,71676.46,71633.41,71636.83,15.90818,1.139815e+06,1209,5.64614,4.045180e+05,71670.898,71724.288,71792.1150,71757.925667,71740.601000,71761.118167,71831.815625,71725.532778,71817.811559,71633.253997
47279,1712620440000,71636.84,71636.84,71617.17,71621.38,24.05450,1.722863e+06,1326,7.46653,5.347731e+05,71655.704,71698.026,71781.9145,71753.704333,71739.714500,71760.329667,71828.855917,71724.106889,71816.997335,71631.216443
47280,1712620620000,71621.38,71662.94,71620.00,71620.00,42.03767,3.011572e+06,1624,17.24372,1.235209e+06,71638.130,71676.880,71767.1100,71750.403667,71738.181333,71759.669417,71825.589250,71721.620889,71814.267279,71628.974499


In [9]:
# Define the window sizes for the moving averages
windows = [5, 10, 20, 30, 60, 120, 240]
# Define the window size and standard deviation multiplier for the Bollinger Bands
BB_window_size = 90
BB_std_multiplier = 1


# ADDING MA
for window in windows:
    # Calculate the moving average
    moving_avg = data_2023_no_opentime['Close'].rolling(window).mean()
    # Add the moving average as a new column to the data_2023_no_opentime
    data_2023_no_opentime[f'MA{window}'] = moving_avg

# Calculate the moving average and standard deviation
BB_moving_avg = data_2023_no_opentime['Close'].rolling(BB_window_size).mean()
BB_std_dev = data_2023_no_opentime['Close'].rolling(BB_window_size).std()

# ADDING BB
# Calculate the Bollinger Bands
BB_upper_band = BB_moving_avg + (BB_std_multiplier * BB_std_dev)
BB_lower_band = BB_moving_avg - (BB_std_multiplier * BB_std_dev)

# Add the Bollinger Bands and moving average as new columns to the data_2023_no_opentime
data_2023_no_opentime[f'MA{BB_window_size}'] = BB_moving_avg
data_2023_no_opentime[f'Upper_Band{BB_window_size}'] = BB_upper_band
data_2023_no_opentime[f'Lower_Band{BB_window_size}'] = BB_lower_band





# drop NaN values
data_2023_no_opentime = data_2023_no_opentime.dropna().reset_index(drop=True)
data_2023_no_opentime.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume',
       'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60',
       'MA120', 'MA240', 'MA90', 'Upper_Band90', 'Lower_Band90'],
      dtype='object')

In [10]:
# Define the window sizes for the moving averages
windows = [5, 10, 20, 30, 60, 120, 240]
# Define the window size and standard deviation multiplier for the Bollinger Bands
BB_window_size = 90
BB_std_multiplier = 1


# ADDING MA
for window in windows:
    # Calculate the moving average
    moving_avg = data_2024_no_opentime['Close'].rolling(window).mean()
    # Add the moving average as a new column to the data_2023_no_opentime
    data_2024_no_opentime[f'MA{window}'] = moving_avg

# Calculate the moving average and standard deviation
BB_moving_avg = data_2024_no_opentime['Close'].rolling(BB_window_size).mean()
BB_std_dev = data_2024_no_opentime['Close'].rolling(BB_window_size).std()

# ADDING BB
# Calculate the Bollinger Bands
BB_upper_band = BB_moving_avg + (BB_std_multiplier * BB_std_dev)
BB_lower_band = BB_moving_avg - (BB_std_multiplier * BB_std_dev)

# Add the Bollinger Bands and moving average as new columns to the data_2023_no_opentime
data_2024_no_opentime[f'MA{BB_window_size}'] = BB_moving_avg
data_2024_no_opentime[f'Upper_Band{BB_window_size}'] = BB_upper_band
data_2024_no_opentime[f'Lower_Band{BB_window_size}'] = BB_lower_band





# drop NaN values
data_2024_no_opentime = data_2024_no_opentime.dropna().reset_index(drop=True)
data_2024_no_opentime.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume',
       'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60',
       'MA120', 'MA240', 'MA90', 'Upper_Band90', 'Lower_Band90'],
      dtype='object')

In [11]:
# Initialize empty lists to hold the data
matrix_list = []


for i in range(len(data_2023) - 79):  # Ensure there are 20 rows available
    # Create a matrix of 20 rows
    matrix = data_2023.iloc[i:i+80].values
    matrix_list.append(matrix)

# Convert the lists to numpy arrays
data_2023_matrix_80 = np.array(matrix_list)


In [12]:
matrix_list = []


for i in range(len(data_2024) - 79):  # Ensure there are 20 rows available
    # Create a matrix of 20 rows
    matrix = data_2024.iloc[i:i+80].values
    matrix_list.append(matrix)

# Convert the lists to numpy arrays
data_2024_matrix_80 = np.array(matrix_list)

In [13]:
# Initialize empty lists to hold the data
matrix_list = []


for i in range(len(data_2023_no_opentime) - 79):  # Ensure there are 20 rows available
    # Create a matrix of 20 rows
    matrix = data_2023_no_opentime.iloc[i:i+80].values
    matrix_list.append(matrix)

# Convert the lists to numpy arrays
data_2023_no_opentime_matrix_80 = np.array(matrix_list)


In [14]:
# Initialize empty lists to hold the data
matrix_list = []


for i in range(len(data_2024_no_opentime) - 79):  # Ensure there are 20 rows available
    # Create a matrix of 20 row
    matrix = data_2024_no_opentime.iloc[i:i+80].values
    matrix_list.append(matrix)

# Convert the lists to numpy arrays
data_2024_no_opentime_matrix_80 = np.array(matrix_list)


In [15]:
data_2023_matrix_80.shape, data_2023_no_opentime_matrix_80.shape, data_2024_matrix_80.shape, data_2024_no_opentime_matrix_80.shape

((174857, 80, 20), (174857, 80, 19), (47203, 80, 20), (47203, 80, 19))

In [17]:
np.save('Data/data_2023_matrix_80.npy', data_2023_matrix_80)
np.save('Data/data_2024_matrix_80.npy', data_2024_matrix_80)

## RFC model

In [18]:
import joblib

# Assuming your model is named RFC
RFC = joblib.load('Models/RFC_model_80.pkl')
scaler = joblib.load('Scalers/StandardScaler_80.pkl')

In [20]:
# scale the data

# Assuming data is your 600k matrices concatenated into a single 3D numpy array of shape (600000, 20, 19)
temp = data_2023_no_opentime_matrix_80.reshape(-1, 19)  # Reshape to 2D for standardization
temp_normalized = scaler.transform(temp)

# Reshape back to 3D
data_2023_matrix_80_normalized = temp_normalized.reshape(-1, 80, 19)

data_2023_matrix_80_normalized_flattened = np.array([x.flatten() for x in data_2023_matrix_80_normalized])

y_pred = RFC.predict(data_2023_matrix_80_normalized_flattened)

np.save('Data/RFC_80_2023_prediction.npy', y_pred)

[Parallel(n_jobs=28)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 100 out of 100 | elapsed:    0.4s finished


In [21]:
# scale the data

# Assuming data is your 600k matrices concatenated into a single 3D numpy array of shape (600000, 20, 19)
temp = data_2024_no_opentime_matrix_80.reshape(-1, 19)  # Reshape to 2D for standardization
temp_normalized = scaler.transform(temp)

# Reshape back to 3D
data_2024_matrix_80_normalized = temp_normalized.reshape(-1, 80, 19)

data_2024_matrix_80_normalized_flattened = np.array([x.flatten() for x in data_2024_matrix_80_normalized])

y_pred = RFC.predict(data_2024_matrix_80_normalized_flattened)

np.save('Data/RFC_80_2024_prediction.npy', y_pred)

[Parallel(n_jobs=28)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 100 out of 100 | elapsed:    0.1s finished
